# Module 1: Tool Calling

**Duration:** ~15 minutes  
**Goal:** Understand why tools matter, build your first agent with the `@tool` decorator

---

## The Core Insight

LLMs are excellent at understanding language and reasoning through problems.  
But they're terrible at **doing things** — fetching data, running calculations, taking actions.

**Tool calling bridges that gap.**

```
┌──────────────────────────────────────────────────┐
│                    THE BRIDGE                     │
│                                                   │
│   USER ──→ LLM ──→ TOOL ──→ REAL DATA ──→ LLM   │
│                                                   │
│   "What's AAPL?"  "I should    get_stock_price   │
│                    call the    returns 178.50    │
│                    price tool"                    │
└──────────────────────────────────────────────────┘
```

## Setup

First, let's install the smolagents library from Hugging Face.

In [1]:
# Install smolagents (the framework we'll use)
!pip install smolagents -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [9]:
# Import the core components
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel
import os

## Configure Your API Key

You'll need an OpenAI API key. You can get one at https://platform.openai.com/api-keys

**Option 1:** Enter it securely (recommended)

In [3]:
import getpass
API_KEY = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


**Option 2:** Use an environment variable (if running locally)

In [4]:
# Uncomment if you have OPENAI_API_KEY set in your environment
# API_KEY = os.getenv("OPENAI_API_KEY")

In [10]:
# Initialize the model
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model initialized!")

Model initialized!


---

## The @tool Decorator

The magic happens with the `@tool` decorator. Let's create our first tool.

The decorator does three important things:
1. **Registers** the function as something the LLM can call
2. **Extracts** the function signature (parameter names and types)
3. **Uses the docstring** to explain to the LLM what this tool does

**Key insight:** The docstring isn't just documentation for humans — it's instructions for the AI.

In [11]:
@tool
def get_stock_price(ticker: str) -> float:
    """
    Get the current price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA')

    Returns:
        The current stock price as a float
    """
    # Simulated prices for demo (no API key needed!)
    prices = {
        "AAPL": 178.50,
        "NVDA": 875.30,
        "MSFT": 378.90,
        "GOOGL": 141.25,
        "AMZN": 178.75
    }
    return prices.get(ticker.upper(), 0.0)

print("Tool created: get_stock_price")
print(f"Test call: get_stock_price('AAPL') = ${get_stock_price('AAPL')}")

Tool created: get_stock_price
Test call: get_stock_price('AAPL') = $178.5


---

## Creating the Agent

Now let's create an agent that can use this tool.

In [12]:
# Create the agent with our tool
agent = CodeAgent(
    tools=[get_stock_price],
    model=model
)

print("Agent created with 1 tool: get_stock_price")

Agent created with 1 tool: get_stock_price


---

## The Magic Moment

Let's run our first query!

In [13]:
result = agent.run("What is Apple's current stock price?")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Apple's current stock price?                                                                            │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_stock_price = get_stock_price(ticker='AAPL')                                                               
  print(f"The current stock price for Apple (AAPL) is: {apple_stock_price}")                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The current stock price for Apple (AAPL) is: 178.5

Out: None

[Step 1: Duration 2.83 seconds| Input tokens: 2,022 | Output tokens: 69]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(178.5)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 178.5

[Step 2: Duration 1.90 seconds| Input tokens: 4,220 | Output tokens: 114]

178.5


**What just happened:**
1. We asked in natural language
2. The agent figured out it needed to call `get_stock_price` with 'AAPL'
3. It got the real number from our tool
4. It formulated a response

**No hallucination. Real data.**

---

## What Happened Under the Hood?

Let's turn on verbose mode to see the agent's reasoning process.

In [14]:
# Create agent with verbose logging
agent_verbose = CodeAgent(
    tools=[get_stock_price],
    model=model,
    verbosity_level=LogLevel.INFO  # Shows the reasoning
)

result = agent_verbose.run("What is Apple's current stock price?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Apple's current stock price?                                                                            │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_stock_price = get_stock_price(ticker='AAPL')                                                               
  print(apple_stock_price)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
178.5

Out: None

[Step 1: Duration 1.46 seconds| Input tokens: 2,022 | Output tokens: 70]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(178.5)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 178.5

[Step 2: Duration 1.29 seconds| Input tokens: 4,194 | Output tokens: 111]

**This transparency is powerful.** You can see exactly why the agent did what it did. No black box.

---

## Adding More Tools

One tool is useful. Multiple tools are powerful. Let's add a few more.

In [15]:
@tool
def get_company_info(ticker: str) -> str:
    """
    Get basic company information for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA')

    Returns:
        A string with company name, sector, and market cap
    """
    companies = {
        "AAPL": "Apple Inc. | Technology | $2.8T market cap",
        "NVDA": "NVIDIA Corporation | Semiconductors | $2.2T market cap",
        "MSFT": "Microsoft Corporation | Technology | $3.1T market cap",
        "GOOGL": "Alphabet Inc. | Technology | $1.9T market cap",
        "AMZN": "Amazon.com Inc. | Consumer Cyclical | $1.9T market cap"
    }
    return companies.get(ticker.upper(), "Company not found")


@tool
def calculate_position_value(ticker: str, shares: int) -> float:
    """
    Calculate the total value of a stock position.

    Args:
        ticker: The stock symbol
        shares: Number of shares owned

    Returns:
        Total position value in dollars
    """
    price = get_stock_price(ticker)
    return price * shares

print("Created 2 additional tools:")
print("- get_company_info")
print("- calculate_position_value")

Created 2 additional tools:
- get_company_info
- calculate_position_value


In [16]:
# Create agent with all three tools
agent = CodeAgent(
    tools=[get_stock_price, get_company_info, calculate_position_value],
    model=model,
    verbosity_level=LogLevel.INFO
)

print("Agent now has 3 tools!")

Agent now has 3 tools!


---

## Multi-Tool Queries

Now the agent can choose which tool to use based on the question.

In [17]:
result = agent.run("Tell me about NVIDIA and what 100 shares would be worth")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about NVIDIA and what 100 shares would be worth                                                         │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_info = get_company_info(ticker='NVDA')                                                                    
  print(f"NVIDIA Company Info: {nvidia_info}")                                                                     
                                                                                                                   
  nvidia_stock_price = get_stock_price(ticker='NVDA')                                                              
  print(f"NVIDIA Current Stock Price: {nvidia_stock_price}")                                                       
                                                                                                                   
  position_value = calculate_position_value(ticker='NVDA', shares=100)                                             
  print(f"Value of 100 NVIDIA shares: {position_value}")                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
NVIDIA Company Info: NVIDIA Corporation | Semiconductors | $2.2T market cap
NVIDIA Current Stock Price: 875.3
Value of 100 NVIDIA shares: 87530.0

Out: None

[Step 1: Duration 2.74 seconds| Input tokens: 2,120 | Output tokens: 146]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "company_info": "NVIDIA Corporation | Semiconductors | $2.2T market cap",                                    
      "current_stock_price": 875.3,                                                                                
      "value_of_100_shares": 87530.0                                                                               
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'company_info': 'NVIDIA Corporation | Semiconductors | $2.2T market cap', 'current_stock_price': 
875.3, 'value_of_100_shares': 87530.0}

[Step 2: Duration 2.37 seconds| Input tokens: 4,581 | Output tokens: 248]

**Two tools, one query!** The agent figured out it needed both pieces of information and called the right tools in the right order.

This is the power of tool calling:
- The LLM handles the **understanding and reasoning**
- The tools handle the **doing**

---

## The Key Principles

Three principles for writing good tools:

### Principle 1: Clear Docstrings
The LLM reads your docstring to decide when to use the tool. Be specific about what it does and when to use it.

### Principle 2: Type Hints Matter
The `ticker: str` and `-> float` tell the LLM what type of data to pass and what to expect back. Don't skip them.

### Principle 3: One Tool, One Job
Don't make tools that do five things. Make five tools that each do one thing well. The agent can combine them.

Every interaction is stored. This is how the agent maintains context across queries.

**Key insight:** Memory is an internal tool, just like `get_stock_price` is an external tool. External tools fetch data from the world. Internal tools manage the agent's state.

**Note:** Memory is session-only. When you create a new agent, it starts fresh. Persistent memory (saving conversations to a database) is covered in the full course.

In [18]:
# Inspect what the agent remembers
print("Agent Memory:")
print("=" * 50)

for i, step in enumerate(agent.memory.steps):
    step_type = type(step).__name__
    print(f"\nStep {i+1}: {step_type}")

    # TaskStep contains the original query
    if step_type == "TaskStep":
        if hasattr(step, 'task'):
            print(f"   Task: {step.task}")

    # ActionStep contains tool calls and results
    elif step_type == "ActionStep":
        if hasattr(step, 'tool_calls') and step.tool_calls:
            for tc in step.tool_calls:
                tool_name = getattr(tc, 'name', 'unknown')
                tool_args = getattr(tc, 'arguments', {})
                print(f"   Tool: {tool_name}({tool_args})")
        if hasattr(step, 'observations') and step.observations:
            obs = str(step.observations)[:60] + "..." if len(str(step.observations)) > 60 else step.observations
            print(f"   Result: {obs}")

Agent Memory:

Step 1: TaskStep
   Task: Tell me about NVIDIA and what 100 shares would be worth

Step 2: ActionStep
   Tool: python_interpreter(nvidia_info = get_company_info(ticker='NVDA')
print(f"NVIDIA Company Info: {nvidia_info}")

nvidia_stock_price = get_stock_price(ticker='NVDA')
print(f"NVIDIA Current Stock Price: {nvidia_stock_price}")

position_value = calculate_position_value(ticker='NVDA', shares=100)
print(f"Value of 100 NVIDIA shares: {position_value}"))
   Result: Execution logs:
NVIDIA Company Info: NVIDIA Corporation | Se...

Step 3: ActionStep
   Tool: python_interpreter(final_answer({
    "company_info": "NVIDIA Corporation | Semiconductors | $2.2T market cap",
    "current_stock_price": 875.3,
    "value_of_100_shares": 87530.0
}))
   Result: Execution logs:
Last output from code snippet:
{'company_inf...


### Conversation Memory

By default, each `agent.run()` call **resets memory** and starts fresh. To maintain context across multiple queries, use `reset=False`:

```python
# First query - starts fresh
agent.run("What's Apple's stock price?")

# Follow-up - preserves memory from previous run
agent.run("And what about NVIDIA?", reset=False)  # ← Key parameter!
```

**Without `reset=False`:** Each query is independent (no conversation)
**With `reset=False`:** Agent remembers previous context (true conversation)

In [19]:
# First question - starts fresh
result = agent.run("What's Apple's stock price?")
print("Query 1:", result)

# Follow-up — use reset=False to PRESERVE memory from previous run
# Without this, each run() starts fresh and loses context
result = agent.run("And what about NVIDIA?", reset=False)
print("\nQuery 2:", result)

# Now the agent should remember we were asking about stock prices!

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's Apple's stock price?                                                                                     │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_stock_price = get_stock_price(ticker='AAPL')                                                               
  print("Apple's stock price:", apple_stock_price)                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Apple's stock price: 178.5

Out: None

[Step 1: Duration 1.46 seconds| Input tokens: 2,113 | Output tokens: 61]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(875.3)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 875.3

[Step 4: Duration 1.22 seconds| Input tokens: 9,318 | Output tokens: 223]


Query 2: 875.3


---

## Internal Tools: Memory

So far we've talked about **external tools** — functions that fetch data from the outside world (`get_stock_price`, `get_company_info`).

But agents also have **internal tools**. The most important one is **memory**.

```
┌─────────────────────────────────────────────────────────────┐
│                      AGENT TOOLS                             │
│                                                              │
│  EXTERNAL TOOLS              INTERNAL TOOLS                  │
│  ┌─────────────────┐        ┌─────────────────┐             │
│  │ get_stock_price │        │     MEMORY      │             │
│  │ get_company_info│        │                 │             │
│  │ calculate_value │        │ Stores context  │             │
│  │                 │        │ Enables recall  │             │
│  │ → Fetch data    │        │ → Continuity    │             │
│  │   from world    │        │   across turns  │             │
│  └─────────────────┘        └─────────────────┘             │
│                                                              │
│  @tool decorator             agent.memory                    │
└─────────────────────────────────────────────────────────────┘
```

Memory stores everything: what you asked, what tools were called, what results came back. It's what lets the agent have a **conversation** rather than just answering one-off questions.

---

## Recap

**What you learned in this module:**

1. **Tool calling** bridges the gap between LLM reasoning and real-world data
2. The **@tool decorator** registers a function for the agent to use
3. **Docstrings and type hints** tell the agent what the tool does and how to use it
4. Agents can use **multiple tools** to answer complex questions
5. **Memory** is an internal tool that maintains context across queries

**Next up:** In Module 2, we'll see what happens when the agent needs to reason through multiple steps. That's the **ReAct pattern**.

In [ ]:
# EXERCISE: Create the get_52_week_high tool

@tool
def get_52_week_high(ticker: str) -> float:
    """
    # YOUR DOCSTRING HERE
    # Describe what the tool does, its args, and return value
    """
    # YOUR CODE HERE
    # Create a dictionary of 52-week highs and return the value
    pass

# Test your tool
# print(f"AAPL 52-week high: ${get_52_week_high('AAPL')}")

In [ ]:
# Once you've created your tool, add it to the agent and test it!

# agent_with_high = CodeAgent(
#     tools=[get_stock_price, get_company_info, calculate_position_value, get_52_week_high],
#     model=model,
#     verbosity_level=LogLevel.INFO
# )

# result = agent_with_high.run("How far is Apple from its 52-week high?")

---

## Solution (Don't peek until you've tried!)

In [ ]:
# SOLUTION

@tool
def get_52_week_high_solution(ticker: str) -> float:
    """
    Get the 52-week high price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA')

    Returns:
        The 52-week high price as a float
    """
    highs = {
        "AAPL": 199.62,
        "NVDA": 974.00,
        "MSFT": 420.82,
        "GOOGL": 155.20,
        "AMZN": 201.20
    }
    return highs.get(ticker.upper(), 0.0)

print(f"AAPL 52-week high: ${get_52_week_high_solution('AAPL')}")

---

## Recap

**What you learned in this module:**

1. **Tool calling** bridges the gap between LLM reasoning and real-world data
2. The **@tool decorator** registers a function for the agent to use
3. **Docstrings and type hints** tell the agent what the tool does and how to use it
4. Agents can use **multiple tools** to answer complex questions

**Next up:** In Module 2, we'll see what happens when the agent needs to reason through multiple steps. That's the **ReAct pattern**.